In [44]:
%matplotlib inline
import jams
import milsed
import pumpp
import json
import pickle
import os
import numpy as np

Using TensorFlow backend.


In [6]:
# fileid = 'Y-VULyMtKazE_0.000_7.000'
fileid = 'Y---lTs1dxhU_30.000_40.000'
jamfile = os.path.join('/home/js7561/dev/milsed/models/resources/model011/predictions/',
                       fileid + '.jams')
jam = jams.load(jamfile)

In [7]:
jam

,time,duration,value,confidence
0,0.000,10.000,Car,None
,time,duration,value,confidence
0,0.000,0.255,Car,None
1,1.556,7.198,Car,None
2,8.754,0.186,Car,None
3,8.754,0.186,Car passing by,None
4,8.940,1.068,Car,None
,time,duration,value,confidence
0,0.000,10.000,Car,1.0
1,0.000,10.000,Car passing by,1.0


In [8]:
jam_trim = jam.trim(0, 5, strict=False)

In [9]:
jam_trim

<JAMS(file_metadata=<FileMetadata(...)>,
      annotations=[3 annotations],
      sandbox=<Sandbox(...)>)>

In [12]:
jam_trim.annotations.search(annotation_tools='dynamic')

[1 annotation]

In [18]:
d = jam_trim.annotations.search(annotation_tools='reference')

In [19]:
len(d)

1

In [20]:
type(d)

jams.core.AnnotationArray

In [21]:
type(d[0])

jams.core.Annotation

## Ensure pump converts static labels correctly

In [4]:
import pickle
import numpy as np

In [3]:
pump = pickle.load(open('/home/js7561/dev/milsed/models/resources/pump.pkl', 'rb'))

In [7]:
duration = 10.0

In [6]:
spred = np.zeros(17)
spred[0] = 0.6
spred = np.asarray([spred])
spred

array([[ 0.6,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ]])

In [8]:
pump['static'].inverse(spred[0], duration=duration)

,time,duration,value,confidence
0,0.000,10.000,"Air horn, truck horn",None


In [9]:
spred = np.zeros(17)
spred[0] = 0.5
spred = np.asarray([spred])
pump['static'].inverse(spred[0], duration=duration)

,time,duration,value,confidence
0,0.000,10.000,"Air horn, truck horn",None


In [10]:
spred = np.zeros(17)
spred[0] = 0.4999
spred = np.asarray([spred])
pump['static'].inverse(spred[0], duration=duration)

,time,duration,value,confidence


## Convert start/dynamic predictions to DCASE format

In [21]:
import jams
import pumpp

In [19]:
jamfile = '/home/js7561/dev/milsed/models/resources/model015/predictions/Y--0w1YA1Hm4_30.000_40.000.jams'
jam = jams.load(jamfile)

In [20]:
jam

<JAMS(file_metadata=<FileMetadata(...)>,
      annotations=[3 annotations],
      sandbox=<Sandbox(...)>)>

In [25]:
ann_s = jam.annotations.search(annotation_tools='static')[0]
ann_d = jam.annotations.search(annotation_tools='dynamic')[0]

In [27]:
df_d = ann_d.to_dataframe()
df_d

,time,duration,value,confidence
0,0.000000,0.301859,Car,None
1,1.277098,2.879274,Car,None
2,5.712109,4.287891,Car,None


In [28]:
df_d['filename'] = 'audio/test_dynamic.wav'
df_d

,time,duration,value,confidence,filename
0,0.000000,0.301859,Car,None,audio/test_dynamic.wav
1,1.277098,2.879274,Car,None,audio/test_dynamic.wav
2,5.712109,4.287891,Car,None,audio/test_dynamic.wav


In [29]:
df_d['start_time'] = df_d.time
df_d

,time,duration,value,confidence,filename,start_time
0,0.000000,0.301859,Car,None,audio/test_dynamic.wav,0.000000
1,1.277098,2.879274,Car,None,audio/test_dynamic.wav,1.277098
2,5.712109,4.287891,Car,None,audio/test_dynamic.wav,5.712109


In [30]:
df_d['end_time'] = df_d.time + df_d.duration
df_d

,time,duration,value,confidence,filename,start_time,end_time
0,0.000000,0.301859,Car,None,audio/test_dynamic.wav,0.000000,0.301859
1,1.277098,2.879274,Car,None,audio/test_dynamic.wav,1.277098,4.156372
2,5.712109,4.287891,Car,None,audio/test_dynamic.wav,5.712109,10.000000


In [31]:
df_d['label'] = df_d['value']
df_d

,time,duration,value,confidence,filename,start_time,end_time,label
0,0.000000,0.301859,Car,None,audio/test_dynamic.wav,0.000000,0.301859,Car
1,1.277098,2.879274,Car,None,audio/test_dynamic.wav,1.277098,4.156372,Car
2,5.712109,4.287891,Car,None,audio/test_dynamic.wav,5.712109,10.000000,Car


In [32]:
df_d_ordered = df_d[['filename', 'start_time', 'end_time', 'label']]
df_d_ordered

,filename,start_time,end_time,label
0,audio/test_dynamic.wav,0.000000,0.301859,Car
1,audio/test_dynamic.wav,1.277098,4.156372,Car
2,audio/test_dynamic.wav,5.712109,10.000000,Car


In [34]:
fid = 'test_static'
# PROCESS STATIC LABELS
df_s = ann_s.to_dataframe()
df_s['filename'] = 'audio/{}.wav'.format(fid)
df_s['start_time'] = df_s.time
df_s['end_time'] = df_s.time + df_s.duration
df_s['label'] = df_s['value']
df_s_ordered = df_s[['filename', 'start_time', 'end_time', 'label']]

In [35]:
df_s_ordered

,filename,start_time,end_time,label
0,audio/test_static.wav,0.0,10.0,Car


In [36]:
df_d_append = df_d_ordered.append(df_d_ordered)

In [37]:
df_d_ordered

,filename,start_time,end_time,label
0,audio/test_dynamic.wav,0.000000,0.301859,Car
1,audio/test_dynamic.wav,1.277098,4.156372,Car
2,audio/test_dynamic.wav,5.712109,10.000000,Car


In [38]:
df_d_append

,filename,start_time,end_time,label
0,audio/test_dynamic.wav,0.000000,0.301859,Car
1,audio/test_dynamic.wav,1.277098,4.156372,Car
2,audio/test_dynamic.wav,5.712109,10.000000,Car
0,audio/test_dynamic.wav,0.000000,0.301859,Car
1,audio/test_dynamic.wav,1.277098,4.156372,Car
2,audio/test_dynamic.wav,5.712109,10.000000,Car


In [39]:
df_d_append.to_csv('~/test.txt', header=False, index=False, sep='\t')

## Some anomalies with static labels, investigate!

In [46]:
OUTPUT_PATH = '/home/js7561/dev/milsed/models/resources/'

In [40]:
jamfile = '/home/js7561/dev/milsed/models/resources/model011/predictions_eval/YeyFPHlybqDg_30.000_40.000.jams'
jam = jams.load(jamfile)

In [41]:
jam

<JAMS(file_metadata=<FileMetadata(...)>,
      annotations=[2 annotations],
      sandbox=<Sandbox(...)>)>

In [42]:
ann_s = jam.annotations.search(annotation_tools='static')[0]
ann_s

<Annotation(namespace='tag_open',
            time=0,
            duration=10.0,
            annotation_metadata=<AnnotationMetadata(...)>,
            data=<0 observations>,
            sandbox=<Sandbox(...)>)>

In [47]:
durfile = os.path.join(OUTPUT_PATH, 'durations.json')
durations = json.load(open(durfile, 'r'))

In [49]:
fid = 'YeyFPHlybqDg_30.000_40.000'
durations[fid]

10.0

In [50]:
df_s = ann_s.to_dataframe()
df_s['filename'] = 'audio/{}.wav'.format(fid)
df_s['start_time'] = df_s.time
df_s['end_time'] = df_s.time + df_s.duration
df_s['label'] = df_s['value']
df_s_ordered = df_s[['filename', 'start_time', 'end_time', 'label']]

In [51]:
df_s_ordered

,filename,start_time,end_time,label


In [52]:
df_s

,time,duration,value,confidence,filename,start_time,end_time,label


Found bug (was saving dynamic into static), fixed!